In [23]:
from functools import wraps
from types import MethodType

decorator function

In [2]:
def logger(fn):
    
    @wraps(fn)
    def wrapped(*args, **kwargs):
        print(f"Log: {fn.__name__} called.")
        return fn(*args, **kwargs)
    return wrapped

In [3]:
@logger
def say_hello():
    pass

In [4]:
say_hello()

Log: say_hello called.


decorator class

In [6]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f"Log: {self.fn.__name__} called.")
        return self.fn(*args, **kwargs)

In [11]:
def say_hello():
    return "hello"

In [12]:
f = Logger(say_hello)

In [13]:
f()

Log: say_hello called.


'hello'

In [14]:
@Logger
def say_hello():
    return "hello"

In [15]:
say_hello()

Log: say_hello called.


'hello'

say_hello is not a function anymore, it's an instance of a class Logger

and that leads us to an issue

In [16]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f"{self.name} says hello!"

In [17]:
p = Person("David")

In [18]:
p.say_hello()

Log: say_hello called.


TypeError: say_hello() missing 1 required positional argument: 'self'

logger ran, than it called the function, but self wasn't pass to function we are decorating. but we called it from the instance. why p wasn't passed to say_hello? say_hello has been decorated, so when the class Person is being created, say_hello is an instance of a class Logger, not a function, so it doesn't get transformed into a method, when we call it from an instance (functions implement non-data descriptor protocol).Logger class does not implement descriptor protocol

In [19]:
hasattr(Person.__init__, "__get__")

True

In [21]:
hasattr(Person.say_hello, "__get__")

False

we can implement `__get__` method in Logger class to turn it into a non-data descriptor and fix the problem

In [24]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f"Log: {self.fn.__name__} called.")
        return self.fn(*args, **kwargs)
    
    def __get__(self, instance, owner_class):
        print(f'__get__ called: self={self}, instance={instance}')
        if instance is None:
            print("\treturning self unbound...")
            return self
        else:
            print("\treturning self as a method bound to the instance...")
            return MethodType(self, instance) # creating bound method: 
        # self is callable (we implemented __call__), __call__ returns function,
        # and we bind it to the instance - so we have a bound method
        

In [25]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f"{self.name} says hello!"

In [26]:
p = Person("David")

In [27]:
p.say_hello()

__get__ called: self=<__main__.Logger object at 0x00000184370FBC88>, instance=<__main__.Person object at 0x00000184370FB9C8>
	returning self as a method bound to the instance...
Log: say_hello called.


'David says hello!'

In [28]:
@Logger
def say_bye():
    pass

In [29]:
say_bye() # this way python wasn't looking for descriptor. 
#it's a callable, so python called a call method

Log: say_bye called.


does our Logger work with classmethods and staticmethods?

In [32]:
class Person:
    
    @classmethod
    @Logger
    def cls_method(cls):
        print("class method called")
        
     
    @staticmethod
    @Logger
    def stat_method():
        print("static method called")

In [31]:
Person.cls_method()

Log: cls_method called.
static method called


In [33]:
Person.stat_method()

Log: stat_method called.
static method called
